This is a live notebook with experimental code to develop functionality for grouping and cross validation.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'

['featured_eeg_prediction_random_forest.ipynb', 'feature_development.ipynb', 'data', 'data_and_processing_description.ipynb', 'featured_eeg_prediction.ipynb', 'gait_data_exploration.ipynb', 'featured_eeg_prediction_gradient_boosting.ipynb', 'Analyse_results.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'featured_E4_prediction.ipynb', 'data_investigation_scratch.ipynb', 'grouping_and_crossvalidation.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")


thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH

from prepare_evaluation_data import load_dataframe_and_config


from predicament.evaluation.results import output_model_best_from_results
from predicament.evaluation.results import save_results_df_to_file


ImportError: cannot import name 'load_dataframe_and_config' from 'prepare_evaluation_data' (/home/luke/git/external/predicament/prepare_evaluation_data.py)

## Column name changes

Changing the names of certain columns after datasets have been produced.

In [ ]:
import os

# from pathlib import Path

# pathlist = Path(WINDOWED_BASE_PATH).rglob('*/*.csv')
# for path in pathlist:
#     # because path is object not string
#     path_in_str = str(path)
#     print(path_in_str)

old_subj_col = 'part_ID'
new_subj_col = 'participant'
col_mapper = {old_subj_col:new_subj_col}
for subdir in os.listdir(WINDOWED_BASE_PATH):
#     filepath = os.path.join(WINDOWED_BASE_PATH, filename)
    windowed_data_dir = os.path.join(WINDOWED_BASE_PATH,subdir)
    if os.path.isdir(windowed_data_dir) and (subdir == 'dummy'):
        windoweddf, windowedconfig = load_dataframe_and_config(
            windowed_data_dir, 'windowed.csv')
        print(f"{subdir}:")
        print(f"- windoweddf.columns = {windoweddf.columns}")
        windoweddf = windoweddf.rename(columns=col_mapper)
        print(f"+ windoweddf.columns = {windoweddf.columns}")
        print(f"- windowedconfig['WINDOWED']['group_col'] = {windowedconfig['WINDOWED']['group_col']}")
        windowedconfig['WINDOWED']['group_col'] = new_subj_col
        print(f"+ windowedconfig['WINDOWED']['group_col'] = {windowedconfig['WINDOWED']['group_col']}")
        print(f"- windowedconfig['WINDOWED']['label_cols'] = {windowedconfig['WINDOWED']['label_cols']}")
        label_cols = json.loads(windowedconfig['WINDOWED']['label_cols'].replace("'",'"'))
#         print(f"label_cols = {label_cols}")
#         print(f"type(label_cols) = {type(label_cols)}")
        part_ID_idx = label_cols.index(old_subj_col)
        label_cols[part_ID_idx] = new_subj_col
#         print(f"label_cols = {label_cols}")
        windowedconfig['WINDOWED']['label_cols'] = str(label_cols)
        print(f"+ windowedconfig['WINDOWED']['label_cols'] = {windowedconfig['WINDOWED']['label_cols']}")


In [ ]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df.rename(columns={"A": "a", "B": "c"})

In [ ]:
windoweddf = windoweddf.rename(columns={'part_ID': 'participant'})
windoweddf['participant']